<a href="https://colab.research.google.com/github/ayushbaghel01/InvestX_asgnmnt/blob/main/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [382]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
#from datetime import datetime

**NOTE : All the code is same as that is provided in the google meet, The code for finding efficiency of the below model is included in the last code blocks.**

In [383]:
df = pd.read_csv('Train.csv')
print(df.head())
df=df.head(150)

         Date    Stock  liabilities      equity  total_assets  current_assets  \
0  2006-07-31  Stock 1   625.898686  925.086518   1550.985204      675.789905   
1  2006-08-31  Stock 1   407.387467  903.249093   1310.636561     1023.110983   
2  2006-09-29  Stock 1   464.069891  602.668832   1066.738722      709.711673   
3  2006-10-31  Stock 1   855.854937  391.375369   1247.230306      649.979564   
4  2006-11-30  Stock 1   453.954969  956.490566   1410.445534     1115.594551   

   current_liabilities  total_revenue  net_income   dividend  \
0           555.931329     548.488451  251.282553  68.184030   
1          1295.186240     513.446432  207.576940  42.232672   
2           598.719338     503.519871  133.719700  28.835722   
3           733.594339     596.006397  -76.881818   0.000000   
4          1152.274711     965.939247 -309.204301   0.000000   

   shares_outstanding       price  
0             1040831  100.000000  
1             1046649  101.924169  
2             109415

In [384]:
df['MACD_line']= (df.price.ewm(span=5).mean() - df.price.ewm(span=13).mean())

signal_line = df.MACD_line.ewm(span = 18).mean()
df['liabilities']=signal_line
df['equity']= df['MACD_line']
df['dividend'] = df.price.ewm(span=14).mean()
df['total_revenue']=(df.MACD_line -signal_line)
print(df)


           Date    Stock  liabilities    equity  total_assets  current_assets  \
0    2006-07-31  Stock 1     0.000000  0.000000   1550.985204      675.789905   
1    2006-08-31  Stock 1     0.062494  0.118410   1310.636561     1023.110983   
2    2006-09-29  Stock 1     0.131864  0.249466   1066.738722      709.711673   
3    2006-10-31  Stock 1     0.216612  0.420989   1247.230306      649.979564   
4    2006-11-30  Stock 1     0.270542  0.435161   1410.445534     1115.594551   
..          ...      ...          ...       ...           ...             ...   
145  2018-08-31  Stock 1    -0.322183 -0.692527   1552.537263      661.976873   
146  2018-09-28  Stock 1    -0.385699 -0.925590   1003.487923      774.901898   
147  2018-10-31  Stock 1    -0.437392 -0.876781   1026.406473      583.243894   
148  2018-11-30  Stock 1    -0.499590 -1.028273   1443.744070      636.300679   
149  2018-12-31  Stock 1    -0.573092 -1.197860   1455.586484      619.734902   

     current_liabilities  t

In [385]:
cols = list(df)[2:12]
print(cols)

['liabilities', 'equity', 'total_assets', 'current_assets', 'current_liabilities', 'total_revenue', 'net_income', 'dividend', 'shares_outstanding', 'price']


In [386]:
df_for_training = df[cols].astype(float)

In [387]:
scaler = StandardScaler()
scaler = scaler.fit(df_for_training)
df_for_training_scaled = scaler.transform(df_for_training)

In [388]:
trainX = []
trainY = []

In [389]:
n_future = 1  # Number of days we want to look into the future based on the past days.
n_past = 14 # Number of past days we want to use to predict the future.

In [390]:
for i in range(n_past, len(df_for_training_scaled) - n_future +1):
    trainX.append(df_for_training_scaled[i - n_past:i, 0:df_for_training.shape[1]])
    trainY.append(df_for_training_scaled[i + n_future - 1:i + n_future, 9])

trainX, trainY = np.array(trainX), np.array(trainY)

print('trainX shape == {}.'.format(trainX.shape))
print('trainY shape == {}.'.format(trainY.shape))

trainX shape == (136, 14, 10).
trainY shape == (136, 1).


In [391]:
df_for_training_scaled;

In [392]:
trainY;

In [393]:
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True)) 
model.add(LSTM(32, activation='relu', return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(trainY.shape[1]))

model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_32 (LSTM)              (None, 14, 64)            19200     
                                                                 
 lstm_33 (LSTM)              (None, 32)                12416     
                                                                 
 dropout_16 (Dropout)        (None, 32)                0         
                                                                 
 dense_16 (Dense)            (None, 1)                 33        
                                                                 
Total params: 31,649
Trainable params: 31,649
Non-trainable params: 0
_________________________________________________________________


In [394]:
history = model.fit(trainX, trainY, epochs=5, batch_size=3, validation_split=0.1, verbose=1)

Epoch 1/5
41/41 [==============================] - 5s 38ms/step - loss: 0.3169 - val_loss: 0.0779
Epoch 2/5
41/41 [==============================] - 1s 21ms/step - loss: 0.1065 - val_loss: 0.0847
Epoch 3/5
41/41 [==============================] - 1s 16ms/step - loss: 0.0789 - val_loss: 0.0880
Epoch 4/5
41/41 [==============================] - 1s 16ms/step - loss: 0.0600 - val_loss: 0.0603
Epoch 5/5
41/41 [==============================] - 1s 16ms/step - loss: 0.0591 - val_loss: 0.0577


In [395]:
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())

In [396]:
 df2= df.head(150)
train_dates = pd.to_datetime(df2['Date'])

In [397]:
n_past = 1
n_days_for_prediction=50  #let us predict past 15 days

predict_period_dates = pd.date_range(list(train_dates)[-n_past], periods=n_days_for_prediction, freq='M').tolist()
print(predict_period_dates)

[Timestamp('2018-12-31 00:00:00', freq='M'), Timestamp('2019-01-31 00:00:00', freq='M'), Timestamp('2019-02-28 00:00:00', freq='M'), Timestamp('2019-03-31 00:00:00', freq='M'), Timestamp('2019-04-30 00:00:00', freq='M'), Timestamp('2019-05-31 00:00:00', freq='M'), Timestamp('2019-06-30 00:00:00', freq='M'), Timestamp('2019-07-31 00:00:00', freq='M'), Timestamp('2019-08-31 00:00:00', freq='M'), Timestamp('2019-09-30 00:00:00', freq='M'), Timestamp('2019-10-31 00:00:00', freq='M'), Timestamp('2019-11-30 00:00:00', freq='M'), Timestamp('2019-12-31 00:00:00', freq='M'), Timestamp('2020-01-31 00:00:00', freq='M'), Timestamp('2020-02-29 00:00:00', freq='M'), Timestamp('2020-03-31 00:00:00', freq='M'), Timestamp('2020-04-30 00:00:00', freq='M'), Timestamp('2020-05-31 00:00:00', freq='M'), Timestamp('2020-06-30 00:00:00', freq='M'), Timestamp('2020-07-31 00:00:00', freq='M'), Timestamp('2020-08-31 00:00:00', freq='M'), Timestamp('2020-09-30 00:00:00', freq='M'), Timestamp('2020-10-31 00:00:00'

In [398]:
prediction = model.predict(trainX[-n_days_for_prediction:]) 

2/2 [==============================] - 0s 8ms/step


In [399]:
prediction_copies = np.repeat(prediction, df_for_training.shape[1], axis=-1)
y_pred_future = scaler.inverse_transform(prediction_copies)[:,9]

In [400]:
forecast_dates = []
for time_i in predict_period_dates:
    forecast_dates.append(time_i.date())
    
df_forecast = pd.DataFrame({'Date':np.array(forecast_dates), 'price':y_pred_future})
df_forecast['Date']=pd.to_datetime(df_forecast['Date'])

In [401]:
original = df[['Date', 'price']]
original['Date']=pd.to_datetime(original['Date'])


<ipython-input-401-d6bbd2f4d398>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  original['Date']=pd.to_datetime(original['Date'])


In [402]:
df_forecast
df_ = pd.read_csv('Train.csv')
df_=df_.head(200)
predicted_price = df_forecast.price

In [403]:
actual_price = df_.price[150:200]

In [404]:
l=[]
for i in range(50):
  l.append(abs(predicted_price[i]-actual_price[150+i])/actual_price[150+i])
deviation_rd=np.array(l)
rms = np.sqrt(np.mean(np.square(deviation_rd)))
efficiency = (1-rms)*100
print(efficiency)
print(l)

84.40160827055145
[0.24808128356933593, 0.21905999131717066, 0.2012954164782627, 0.20780730141924414, 0.20869259222116257, 0.2143158079955871, 0.20817014978404466, 0.21118524070886202, 0.20116617606442272, 0.19182456693490899, 0.18059007590288562, 0.16223792400100104, 0.14433075228053646, 0.13211171425604154, 0.13145216919230115, 0.13423965263861695, 0.13451732368505268, 0.12211130115631977, 0.12965097111114474, 0.13969925499306132, 0.13498001591151507, 0.137183827830456, 0.14977958913711817, 0.14530894677336162, 0.1328770600258682, 0.11359941190540605, 0.1147672539488667, 0.12814162880755717, 0.12001911192723892, 0.1103998778293463, 0.11671711382559638, 0.10857278559111562, 0.11857118160712339, 0.11209866994275183, 0.12137691036661456, 0.1167378241293498, 0.1114742690280042, 0.10039525672834845, 0.10998584016760826, 0.1261541631812187, 0.1351396727309151, 0.13100913893825747, 0.12872270840003863, 0.14083869436101765, 0.15075630417991176, 0.15974538338070196, 0.17646185146757098, 0.192